In [62]:
import random

class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    def __init__(self, height, width):
        """
        Constructeur d'un labyrinthe de height cellules de haut 
        et de width cellules de large 
        Les voisinages sont initialisés à des ensembles vides
        Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
        """
        self.height    = height
        self.width     = width
        self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}

    def info(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = "**Informations sur le labyrinthe**\n"
        txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
        txt += "- Voisinages :\n"
        txt += str(self.neighbors)+"\n"
        valid = True
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in self.neighbors[c1]:
                if c1 not in self.neighbors[c2]:
                    valid = False
                    break
            else:
                continue
            break
        txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
        return txt

    def __str__(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt
    
    def add_wall(self, c1, c2):
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Ajout du mur
        if c2 in self.neighbors[c1]:      # Si c2 est dans les voisines de c1
            self.neighbors[c1].remove(c2) # on le retire
        if c1 in self.neighbors[c2]:      # Si c3 est dans les voisines de c2
            self.neighbors[c2].remove(c1) # on le retire
            
    def remove_wall(self, c1, c2) :     
        self.neighbors[c1].add(c2)       
        self.neighbors[c2].add(c1) 
        
    def get_cells(self):
        cellules = []
        for i in range(0,self.height):
            for j in range(0,self.width):
                cellules.append((i,j))
        return cellules
         
    def get_walls(self):
        walls = []
        for c1 in self.get_cells():
            c2 = (c1[0],c1[1]+1)
            c3 = (c1[0]+1,c1[1])
            if c1[1]+1 < self.width and c2 not in self.neighbors[c1]:
                walls.append((c1,c2))
            if c1[0]+1 < self.height and c3 not in self.neighbors[c1]:
                walls.append((c1,c3))
        return walls
    
    def fill(self):
        for c1 in self.get_cells():
            c2 = (c1[0],c1[1]+1)
            c3 = (c1[0]+1,c1[1])
            if c2 in self.neighbors[c1]:
                self.add_wall(c1,c2)
            if c3 in self.neighbors[c1]:
                self.add_wall(c1,c3)
    
    def empty(self):
        for c1 in self.get_cells():
            c2 = (c1[0], c1[1] + 1)
            c3 = (c1[0] + 1, c1[1])
            if c1[1]+1 < self.width:
                self.remove_wall(c1, c2)
            if c1[0]+1 < self.height:
                self.remove_wall(c1, c3)
                
    def get_contiguous_cells(self, c):
        listeContigue = []
        if c[1] - 1 >= 0:
            listeContigue.append((c[0], c[1] - 1))
        if c[1] + 1 < self.width:
            listeContigue.append((c[0], c[1] + 1))
        if c[0] - 1 >= 0:
            listeContigue.append((c[0] - 1, c[1]))
        if c[0] + 1 < self.height:
            listeContigue.append((c[0] + 1, c[1]))
        return listeContigue 
    
    def get_reachable_cells(self, c):
        listeAccessible = []
        for c1 in self.get_contiguous_cells(c):
            if c1 in self.neighbors[c]:
                listeAccessible.append(c1)
        return listeAccessible

    @classmethod
    def gen_btree(cls, h: int, w: int):
        cls = Maze(h, w)
        cls.fill()
        cEst = None
        cSud = None
        for cellule in cls.get_cells():
            if cellule[1]+1 < w:
                cEst = (cellule[0], cellule[1]+1)
            if cellule[0]+1 < h:
                cSud = (cellule[0]+1, cellule[1])
                
            if cEst == (cellule[0], cellule[1]+1) and \
                    cSud == (cellule[0]+1, cellule[1]):
                if random.randint(1,2) == 1:
                    cls.remove_wall(cellule, cEst)
                else:
                    cls.remove_wall(cellule, cSud)
                    
            if (cEst == (cellule[0], cellule[1]+1)) ^ \
                    (cSud == (cellule[0]+1, cellule[1])):
                if cEst == (cellule[0], cellule[1]+1):
                    cls.remove_wall(cellule, cEst)
                if cSud == (cellule[0]+1, cellule[1]):
                    cls.remove_wall(cellule, cSud)
        return cls
    
    @classmethod
    def gen_sidewinder(cls, h: int, w: int):
        cls = Maze(h, w)
        cls.fill()
        for i in range(h-1):
            sequence = []
            for j in range(w-1):
                sequence.append((i, j))
                choix = random.randint(1, 2)
                if choix == 1:
                    cls.remove_wall(sequence[-1], (sequence[-1][0], sequence[-1][1]+1))
                else:
                    choisit = random.randint(0, len(sequence)-1)
                    cls.remove_wall(sequence[choisit], (sequence[choisit][0]+1, sequence[choisit][1]))
                    sequence = []
            sequence.append((i, j+1))
            choisitFin = random.randint(0, len(sequence)-1)
            cls.remove_wall(sequence[choisitFin], (sequence[choisitFin][0]+1, sequence[choisitFin][1]))
        for cellule in range(w-1):
            cls.remove_wall((h-1, cellule), (h-1, cellule+1))
        return cls 
    
    @classmethod
    def gen_fusion(cls,h: int,w: int):
        cls = Maze(h, w)
        cls.fill()
        labels = []
        for i in range(h - 1):
            for j in range(w - 1):
                labels.append(i * w + j)
        walls = cls.get_walls()
        random.shuffle(walls)
        for wall in range(len(walls)): 
            c1 = walls[wall][0]
            c2 = walls[wall][1]
            label_c1 = labels[c1[0] * w + c1[1]]
            label_c2 = labels[c2[0] * w + c2[1]]
            if label_c1 != label_c2 :
                cls.remove_wall(c1, c2)
                labels[label_c2] = label_c1
                for label in range(len(labels)):
                    if labels[label] == label_c2:
                        labels[label] = label_c1               
        return cls
    
    @classmethod
    def gen_exploration(cls, h: int, w: int):
        cls = Maze(h, w)
        cls.fill()
        voisins = cls.neighbors
        cellules = cls.get_cells()
        cHasard = cellules[random.randint(0, len(cellules) - 1)]
        marques = [False]*(len(cellules)-1)
        marques[cHasard[0] * w + cHasard[1]] = True
        pile = [cHasard]
        voisinsNonVisite = []
        while len(pile) > 0:
            voisins = cls.neighbors
            cellule = pile[-1]
            del pile[-1]
            for voisin in voisins[cellule]:
                if marques[voisin[0] * w + voisin[1]] == False:
                    voisinsNonVisite.append(voisin)
            if len(voisinsNonVisite) > 0:
                pile.append(cellule)
                choix = random.choice(voisinsNonVisite)
                cls.remove_wall(cellule, choix)
                marques[choix[0] * w + choix[1]] = True
                pile.append(choix)
        return cls
                    
                    
                
                    

In [63]:
laby = Maze(4, 4)
print(laby.info())

**Informations sur le labyrinthe**
- Dimensions de la grille : 4 x 4
- Voisinages :
{(0, 0): set(), (0, 1): set(), (0, 2): set(), (0, 3): set(), (1, 0): set(), (1, 1): set(), (1, 2): set(), (1, 3): set(), (2, 0): set(), (2, 1): set(), (2, 2): set(), (2, 3): set(), (3, 0): set(), (3, 1): set(), (3, 2): set(), (3, 3): set()}
- Structure cohérente



In [64]:
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [65]:
laby.neighbors = {
    (0, 0): {(1, 0)},
    (0, 1): {(0, 2), (1, 1)},
    (0, 2): {(0, 1), (0, 3)},
    (0, 3): {(0, 2), (1, 3)},
    (1, 0): {(2, 0), (0, 0)},
    (1, 1): {(0, 1), (1, 2)},
    (1, 2): {(1, 1), (2, 2)},
    (1, 3): {(2, 3), (0, 3)},
    (2, 0): {(1, 0), (2, 1), (3, 0)},
    (2, 1): {(2, 0), (2, 2)},
    (2, 2): {(1, 2), (2, 1)},
    (2, 3): {(3, 3), (1, 3)},
    (3, 0): {(3, 1), (2, 0)},
    (3, 1): {(3, 2), (3, 0)},
    (3, 2): {(3, 1)},
    (3, 3): {(2, 3)}
}

print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [66]:
laby.neighbors[(1,3)].remove((2,3))
laby.neighbors[(2,3)].remove((1,3))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [67]:
laby.neighbors[(1, 3)].add((2, 3))
laby.neighbors[(2, 3)].add((1, 3))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [68]:
laby.neighbors[(1, 3)].remove((2, 3))
print(laby)
print(laby.info())

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛

**Informations sur le labyrinthe**
- Dimensions de la grille : 4 x 4
- Voisinages :
{(0, 0): {(1, 0)}, (0, 1): {(1, 1), (0, 2)}, (0, 2): {(0, 1), (0, 3)}, (0, 3): {(0, 2), (1, 3)}, (1, 0): {(2, 0), (0, 0)}, (1, 1): {(0, 1), (1, 2)}, (1, 2): {(1, 1), (2, 2)}, (1, 3): {(0, 3)}, (2, 0): {(1, 0), (2, 1), (3, 0)}, (2, 1): {(2, 0), (2, 2)}, (2, 2): {(1, 2), (2, 1)}, (2, 3): {(3, 3), (1, 3)}, (3, 0): {(3, 1), (2, 0)}, (3, 1): {(3, 2), (3, 0)}, (3, 2): {(3, 1)}, (3, 3): {(2, 3)}}
- Structure incohérente : (2, 3) X (1, 3)



In [69]:
laby.neighbors[(2, 3)].remove((1,3))

In [70]:
c1 = (1, 3)
c2 = (2, 3)
if c1 in laby.neighbors[c2] and c2 in laby.neighbors[c1]:
    print(f"Il n'y a pas de mur entre {c1} et {c2} car elles sont mutuellement voisines")
elif c1 not in laby.neighbors[c2] and c2 not in laby.neighbors[c1]:
    print(f"Il y a un mur entre {c1} et {c2} car {c1} n'est pas dans le voisinage de {c2} et {c2} n'est pas dans le voisinage de {c1}")
else:
    print(f"Il y a une incohérence de réciprocité des voisinages de {c1} et {c2}")

Il y a un mur entre (1, 3) et (2, 3) car (1, 3) n'est pas dans le voisinage de (2, 3) et (2, 3) n'est pas dans le voisinage de (1, 3)


In [71]:
c1 = (1, 3)
c2 = (2, 3)
if c1 in laby.neighbors[c2] and c2 in laby.neighbors[c1]:
    print(f"{c1} est accessible depuis {c2} et vice-versa")
elif c1 not in laby.neighbors[c2] and c2 not in laby.neighbors[c1]:
    print(f"{c1} n'est pas accessible depuis {c2} et vice-versa")
else:
    print(f"Il y a une incohérence de réciprocité des voisinages de {c1} et {c2}")

(1, 3) n'est pas accessible depuis (2, 3) et vice-versa


In [72]:
L = []
for i in range(laby.height):
    for j in range(laby.width):
        L.append((i,j))
print(f"Liste des cellules : \n{L}")

Liste des cellules : 
[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]


In [73]:
laby = Maze(5, 5)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [74]:
#laby = Maze(5, 5)
#laby.empty()
#print(laby)

In [75]:
laby.add_wall((0,0), (0,1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [76]:
laby.remove_wall((0, 1), (0, 2))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃       ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [77]:
print(laby.get_walls())

[((0, 0), (0, 1)), ((0, 0), (1, 0)), ((0, 1), (1, 1)), ((0, 2), (0, 3)), ((0, 2), (1, 2)), ((0, 3), (0, 4)), ((0, 3), (1, 3)), ((0, 4), (1, 4)), ((1, 0), (1, 1)), ((1, 0), (2, 0)), ((1, 1), (1, 2)), ((1, 1), (2, 1)), ((1, 2), (1, 3)), ((1, 2), (2, 2)), ((1, 3), (1, 4)), ((1, 3), (2, 3)), ((1, 4), (2, 4)), ((2, 0), (2, 1)), ((2, 0), (3, 0)), ((2, 1), (2, 2)), ((2, 1), (3, 1)), ((2, 2), (2, 3)), ((2, 2), (3, 2)), ((2, 3), (2, 4)), ((2, 3), (3, 3)), ((2, 4), (3, 4)), ((3, 0), (3, 1)), ((3, 0), (4, 0)), ((3, 1), (3, 2)), ((3, 1), (4, 1)), ((3, 2), (3, 3)), ((3, 2), (4, 2)), ((3, 3), (3, 4)), ((3, 3), (4, 3)), ((3, 4), (4, 4)), ((4, 0), (4, 1)), ((4, 1), (4, 2)), ((4, 2), (4, 3)), ((4, 3), (4, 4))]


In [78]:
laby = Maze(5,5)
laby.get_cells()

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4)]

In [79]:
laby.remove_wall((0, 1), (0, 2))
print(laby)


┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃       ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [80]:
laby.fill()
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [81]:
laby.empty()
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [82]:
print(laby.get_contiguous_cells((0,0)))

[(0, 1), (1, 0)]


In [83]:
laby.add_wall((0,0),(1,0))
print(laby.get_reachable_cells((0,0)))

[(0, 1)]


In [84]:
print(laby)
print(laby.get_walls())

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃                   ┃
┣━━━╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛

[((0, 0), (1, 0))]


In [85]:
laby = Maze.gen_btree(10,10)

In [86]:
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃                   ┃   ┃       ┃       ┃
┣━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ┫
┃           ┃   ┃               ┃   ┃   ┃
┣━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ┫
┃   ┃   ┃   ┃   ┃                   ┃   ┃
┣   ╋   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ┫
┃   ┃   ┃               ┃   ┃   ┃   ┃   ┃
┣   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ┫
┃   ┃       ┃                   ┃   ┃   ┃
┣   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ┫
┃           ┃   ┃   ┃   ┃           ┃   ┃
┣━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ┫
┃           ┃                   ┃   ┃   ┃
┣━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ┫
┃               ┃       ┃       ┃   ┃   ┃
┣━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ┫
┃               ┃       ┃   ┃           ┃
┣━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ┫
┃                                       ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┛



In [87]:
laby = Maze.gen_sidewinder(10, 10)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃               ┃               ┃   ┃   ┃
┣━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ┫
┃                                   ┃   ┃
┣━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ┫
┃                   ┃   ┃               ┃
┣━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃       ┃   ┃   ┃       ┃
┣   ╋   ╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━┫
┃                       ┃       ┃       ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ┫
┃                       ┃       ┃   ┃   ┃
┣━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ┫
┃                       ┃   ┃   ┃   ┃   ┃
┣━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ┫
┃   ┃           ┃       ┃       ┃   ┃   ┃
┣   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ┫
┃           ┃   ┃               ┃   ┃   ┃
┣━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ┫
┃                                       ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┛



In [88]:
laby = Maze.gen_fusion(15,15)
print(laby)

IndexError: list index out of range

In [90]:
laby = Maze.gen_exploration(10, 10)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┛

